In [2]:
#Importing Data Lesson

In [4]:
!pip install boto3 --upgrade

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [10]:
import sys
import os
import pandas as pd

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util
%reload_ext autoreload
import boto3
import s3fs

In [3]:
region='us-east-1'
bucket_name='shrshk-forecast-tutorial'

In [4]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

In [7]:
role_name = "AmazonSageMaker-ExecutionRole-20210815T200605"
print(f"Fetching Role {role_name} ...")
role_arn = boto3.resource('iam').Role(role_name).arn

# echo user inputs without account
print(f"Success! Retrieved role arn = {role_arn.split('/')[1]}")

Fetching Role AmazonSageMaker-ExecutionRole-20210815T200605 ...
Success! Retrieved role arn = service-role


In [8]:

# check you can communicate with Forecast API session
forecast.list_predictors()

{'Predictors': [],
 'ResponseMetadata': {'RequestId': 'c6c49735-d1d2-4c35-860a-e7c2ce65d51c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 16 Aug 2021 03:53:48 GMT',
   'x-amzn-requestid': 'c6c49735-d1d2-4c35-860a-e7c2ce65d51c',
   'content-length': '17',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [12]:
df = pd.read_csv("../../common/data/item-demand-time.csv", dtype=object, names=['timestamp','value','item'])

In [13]:
df.head(3)

,timestamp,value,item
0,2014-01-01 01:00:00,38.34991708126038,client_12
1,2014-01-01 02:00:00,33.5820895522388,client_12
2,2014-01-01 03:00:00,34.41127694859037,client_12


In [16]:
# Select January to October for one dataframe.
jan_to_oct = df[(df['timestamp'] >= '2014-01-01') & (df['timestamp'] < '2014-11-01')]

print(f"min timestamp = {jan_to_oct.timestamp.min()}")
print(f"max timestamp = {jan_to_oct.timestamp.max()}")

min timestamp = 2014-01-01 01:00:00
max timestamp = 2014-10-31 23:00:00


In [17]:
item_id = 'client_12'

In [19]:
jan_to_oct.to_csv("data/item-demand-time-train.csv", header=False, index=False)

In [21]:
key="elec_data/item-demand-time-train.csv"
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file("data/item-demand-time-train.csv")